# Add Case Base, Cases & Global Similarity Function
This script shows how to add cases to an existing project

In [1]:
# Render our plots inline
%matplotlib inline

from requests import get
import requests
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

from pandas import json_normalize

from mycbr_py_api import MyCBRRestApi as mycbr

In [2]:
server = 'localhost'
#server = 'hv-6066.idi.ntnu.no'

port = '8080'
base_url = 'http://' + server + ':' + port + '/'

obj = mycbr(base_url)

## Adding a new case base and cases to a project

In [3]:
# Existing case bases
obj.getCaseBaseIDs()

['CaseBase0']

In [4]:
obj.addCaseBaseID('NewCB')

True

In [5]:
obj.getCaseBaseIDs()

['NewCB', 'CaseBase0']

In [6]:
# Create a dataframe of cases that should be added
raw_data = pd.read_csv('./data/used_cars.csv')
cases = raw_data.sample(10).reset_index(drop=True) # sample ten cases from the CSV to be added
cases = cases.drop(['caseID'], axis=1) # remove the case ID as we create a new ID in add_rows_as_cases; if you want to keep the ID, adapt add_rows_as_cases accordingly
cases

,Body,CCM,Car Code,Color,Doors,Gas,Manufacturer,Miles,Model,Power,Price,Speed,Year,ZIP
0,station_wagon,2800,486,yellow,5,gasoline,mercedes-benz,136768,c_280,197,19199,213,1995,7


In [7]:
# Finding the name of the concepts
obj.getAllConcepts()

['Car']

In [8]:
headers = {'Content-type':'application/json'}

# add cases to casebase
def add_rows_as_cases(x, concept, casebase):
    case_id = 'case-' + str(x['index'])
    x = x.drop(['index'])
    print(case_id, x.to_json())
    requests.post(base_url + 'concepts/' + concept + '/casebases/' + casebase +'/cases/' + case_id, data = str(x.to_json()), headers=headers)

# add cases to casebase from dataframe
def add_cases_from_df(df, concept, casebase):  
    tmp = df.copy(deep=True)
    tmp.reset_index(inplace=True)
    tmp.apply(add_rows_as_cases, args=(concept, casebase), axis=1)

In [9]:
add_cases_from_df(cases, 'Car', 'NewCB') 

case-0 {"Body":"station_wagon","CCM":2800,"Car Code":486,"Color":"yellow","Doors":5,"Gas":"gasoline","Manufacturer":"mercedes-benz","Miles":136768,"Model":"c_280","Power":197,"Price":19199,"Speed":213,"Year":1995,"ZIP":7}


In [10]:
obj.getAllCasesFromCaseBase('Car', 'NewCB')

,similarity,caseID,Body,CCM,Car Code,Color,Doors,Gas,Manufacturer,Miles,Model,Power,Price,Speed,Year,ZIP
0,1.0,case-0,station_wagon,2800,486,yellow,5,gasoline,mercedes-benz,136768,c_280,197,19199.0,213,1995,7


## Remove Cases

In [11]:
# delete cases from casebase
def delete_all_cases_from_cb(concept, casebase):
    requests.delete(base_url + 'concepts/' + concept + '/casebases/' + casebase +'/cases')

# delete specific case from casebase
def delete_case_from_cb(concept, casebase, caseid):
    requests.delete(base_url + 'concepts/' + concept + '/casebases/' + casebase +'/cases/' + caseid)
    

In [12]:
# Delete all cases
delete_all_cases_from_cb('Car', 'NewCB')

In [13]:
obj.getAllCasesFromCaseBase('Car', 'NewCB')

The response from myCBR is empty! Kindly have a look!
The Dataframe is :  Empty DataFrame
Columns: []
Index: []


""


In [14]:
# add cases again
add_cases_from_df(cases, 'Car', 'NewCB') 

case-0 {"Body":"station_wagon","CCM":2800,"Car Code":486,"Color":"yellow","Doors":5,"Gas":"gasoline","Manufacturer":"mercedes-benz","Miles":136768,"Model":"c_280","Power":197,"Price":19199,"Speed":213,"Year":1995,"ZIP":7}


In [15]:
# show case base
obj.getAllCasesFromCaseBase('Car', 'NewCB')

,similarity,caseID,Body,CCM,Car Code,Color,Doors,Gas,Manufacturer,Miles,Model,Power,Price,Speed,Year,ZIP
0,1.0,case-0,station_wagon,2800,486,yellow,5,gasoline,mercedes-benz,136768,c_280,197,19199.0,213,1995,7


In [16]:
# delete case by case ID
delete_case_from_cb('Car', 'NewCB', 'case-1')

In [17]:
# show case base
obj.getAllCasesFromCaseBase('Car', 'NewCB')

,similarity,caseID,Body,CCM,Car Code,Color,Doors,Gas,Manufacturer,Miles,Model,Power,Price,Speed,Year,ZIP
0,1.0,case-0,station_wagon,2800,486,yellow,5,gasoline,mercedes-benz,136768,c_280,197,19199.0,213,1995,7


## View and add a new amalgamation function

In [18]:
# Existing functions
obj.getAllAmalgamationFunctions('Car')

['CarFunc']

In [19]:
# Method to show the current function
def show_amalgamation_function(concept, amalgamationFunctionID):
    raw = pd.DataFrame(requests.get(base_url + 'analytics/' + 'concepts/' + concept + '/amalgamationFunctions/{amalgamationFunctionID}/globalWeights?amalgamationFunctionID=' + amalgamationFunctionID).json()) .T
    raw.loc[:,'Weights'] = raw.sum(numeric_only=True, axis=1)
    return raw[['Weights']]

# Method to add new amalgamation function
def newAmalgamationFunc(c, amalFuncID, amalFuncType, json):
    return requests.put(base_url + 'concepts/' + c + '/amalgamationFunctions?amalgamationFunctionID=' + amalFuncID + '&amalgamationFunctionType=' + amalFuncType + '&attributeWeightsJSON=' + json)

In [20]:
show_amalgamation_function('Car', 'CarFunc')

,Weights
Body,5.0
CCM,1.0
Car Code,1.0
Color,5.0
Doors,2.0
Gas,1.0
Manufacturer,7.0
Miles,3.0
Model,5.0
Power,3.0


In [21]:
# Define the new function
json = '{"Body":3,"CCM":1,"Car Code":0,"Color":5,"Doors":2,"Gas":1,"Manufacturer":7,"Miles":3,"Model":2,"Power":3,"Price":7,"Speed":1,"Year":1,"ZIP":1}'
# and add it to the concept
newAmalgamationFunc('Car','amal_func_0', 'WEIGHTED_SUM', json)

<Response [200]>

In [22]:
# Show the new function
show_amalgamation_function('Car', 'amal_func_0')

,Weights
Body,3.0
CCM,1.0
Car Code,0.0
Color,5.0
Doors,2.0
Gas,1.0
Manufacturer,7.0
Miles,3.0
Model,2.0
Power,3.0


In [23]:
# Define the new function - only those attributes that are not 1 need to be specified
json = '{"Body":3,"Power":3,"Price":7}'
# and add it to the concept
newAmalgamationFunc('Car','amal_func_1', 'WEIGHTED_SUM', json)

<Response [200]>

In [24]:
# Show the new function
show_amalgamation_function('Car', 'amal_func_1')

,Weights
Body,3.0
CCM,1.0
Car Code,1.0
Color,1.0
Doors,1.0
Gas,1.0
Manufacturer,1.0
Miles,1.0
Model,1.0
Power,3.0
